In [ ]:
import os
os.environ['GROQ_API_KEY'] = 'ygsk_uIuldNtIs91hHpRVnK5wWGdyb3FYkYrlYukZzxVuVw0auqpUqk4l'


In [ ]:
!pip install -q gradio groq openai-whisper gtts python-dotenv

import os
import gradio as gr
from groq import Groq
from gtts import gTTS
import whisper

# Initialize models and clients
# For better speech recognition
model = whisper.load_model("medium")  # More accurate but slower

# Alternative TTS options (add these packages)
!pip install pygame pydub
from pydub import AudioSegment

# Enhanced response formatting
def format_response(response):
    return f"🤖: {response}\n\n(Response generated by LLAMA3-70B)"

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def process_audio(audio_path):
    try:
        # Speech-to-Text with Whisper
        result = model.transcribe(audio_path)
        user_input = result["text"]

        if result['no_speech_prob'] > 0.7:  # Voice activity detection
            return "no_speech_detected.wav", "No speech detected"

        # AI Response Generation
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": user_input}],
            model="llama3-70b-8192"  # Groq's fastest LLAMA3 model
        )
        ai_response = chat_completion.choices[0].message.content

        # Text-to-Speech with gTTS
        tts = gTTS(ai_response, lang='en', tld='us')
        response_path = "response.mp3"
        tts.save(response_path)

        return response_path, ai_response

    except Exception as e:
        return None, f"Error: {str(e)}"

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🎙️ Real-Time Voice Chatbot")

    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak Now")
        audio_output = gr.Audio(label="AI Response", autoplay=True)

    text_output = gr.Textbox(label="Transcript")

    submit_btn = gr.Button("Send")
    submit_btn.click(
        fn=process_audio,
        inputs=audio_input,
        outputs=[audio_output, text_output]
    )

# For Colab deployment
from google.colab import drive
drive.mount('/content/drive')

# Run the interface
demo.launch(debug=True, share=True)


Mounted at /content/drive
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d500202c1f0d79d250.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
